In [1]:
import pandas as pd
import random
import numpy as np
import random
import tensorflow as tf
import operator
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

In [2]:
data=pd.read_csv("../HW-2-Machine-reasoning/dataset1.csv")
data.head(10)

,x,f(x)
0,-0.66,18.37
1,2.99,5.00
2,2.65,5.12
3,-1.72,27.31
4,4.33,6.76
5,1.94,6.13
6,-1.02,21.17
7,0.05,13.73
8,-1.77,27.78
9,1.06,8.76


In [3]:
shuffle(data)

,x,f(x)
9769,1.71,6.65
21903,-1.83,28.36
22307,3.24,5.06
6638,-1.05,21.38
6718,0.98,9.10
...,...,...
16522,0.06,13.62
6471,-0.38,16.45
10786,1.47,7.33
6616,-3.51,47.38


In [4]:
x_train, x_test, y_train, y_test = train_test_split(data['x'],data['f(x)'], train_size=0.80, random_state=0)

In [5]:
x_train = x_train.values
x_test = x_test.values
y_train = y_train.values
y_test = y_test.values

In [6]:
F = ['+','-','*','/']

In [7]:
T = ['x']+ list(range(-5,6))

In [8]:
ran = [F,T]

In [9]:
tf = [True,False]

In [10]:
print(T)

['x', -5, -4, -3, -2, -1, 0, 1, 2, 3, 4, 5]


In [11]:
class Node():
    def __init__(self, *args):
        if(len(args) > 1):
            self.val = args[0]
            self.left = args[1]
            self.right = args[2]
        else:
            self.val = args[0]
            self.left = None
            self.right = None
            
    def insert_left(self, child):
        if (self.left == None):
            self.left = child
        else:
            child.left = self.left
            self.left = child
    def insert_right(self, child):
        if(self.right == None):
            self.right = child
        else:
            child.right = self.right
            self.right = child
    def evaluate(self, x):
        if(self.val in T):
            if(self.val == 'x'):
                return int(x)
            else:
                return int(self.val)
        elif(self.val in F):
            if(self.val == '+'):
                left = self.left.evaluate(x)
                right = self.right.evaluate(x)
                return left + right
            elif(self.val == '-'):
                left = self.left.evaluate(x)
                right = self.right.evaluate(x)
                return left - right
            elif(self.val == '*'):
                left = self.left.evaluate(x)
                right = self.right.evaluate(x)
                return left * right
            elif(self.val == '/'):
                if(self.right.evaluate(x) == 0):
                    return 1
                left = self.left.evaluate(x)
                right = self.right.evaluate(x)
                return left / right
    def fitness(self, x, y):
        return self.evaluate(x)-y
#         return abs(self.evaluate(x)-y)

        

In [12]:

def print2D(root, space) :

    if (root == None) :
        return
    # Increase distance between levels
    space += 10

    print2D(root.right, space)

    # Print current node after space
    print()
    for i in range(10, space):
        print(end = " ")
    print(root.val)

    print2D(root.left, space)



In [13]:
def calculate_mse(x_data, y_data, tree):
    total = 0
    for i in range(len(x_data)):
        total += (tree.fitness(x_data[i],y_data[i]) ** 2)
    return total/len(x_data)

In [14]:
def generate_tree_shapes(size):
    root = Node(random.choice(random.choice(ran)))
    for i in range(size):
        if(random.choice(tf)):
            root.insert_right(Node(random.choice(random.choice(ran))))
            if(random.choice(tf)):
                root.right.insert_right(Node(random.choice(random.choice(ran))))
            if(random.choice(tf)):
                root.right.insert_left(Node(random.choice(random.choice(ran))))
        
        if(random.choice(tf)):
            root.insert_left(Node(random.choice(random.choice(ran))))
            if(random.choice(tf)):
                root.left.insert_right(Node(random.choice(random.choice(ran))))
            if(random.choice(tf)):
                root.left.insert_left(Node(random.choice(random.choice(ran))))

    return root

In [15]:

def generate_tree(size, root):
    if(size == 1):
        root.right = Node(random.choice(T))
        root.left = Node(random.choice(T))
    else:
        #right node
        newNode = random.choice(ran)
        if (newNode == F):
            root.right = Node(random.choice(F))
            generate_tree(size-1, root.right)
        else:
            root.right = Node(random.choice(T))
        
        #left node
        newNode = random.choice(ran)
        if (newNode == F):
            root.left = Node(random.choice(F))
            generate_tree(size-1,root.left)
        else:
            root.left = Node(random.choice(T))
root = Node(random.choice(F))

In [16]:
def mutateHelper():
    mutation = Node(random.choice(F))
    (generate_tree(3, mutation))
    return mutation

def mutate(tree):
    path = []
    path = randomSubTree(tree, path)
    mutation = mutateHelper()
    tree = replaceNode(tree, path, mutation)
    return tree


rl = [-1,1]
tf = [True, True, True, False]
def randomSubTree(node, path):
    keepGoing = random.choice(tf)
    if(keepGoing and len(path) < 3):
        nextNode = random.choice([node.right, node.left])
        if(nextNode == None):
            return path
        if(nextNode == node.right):
            path.append(1)
            randomSubTree(node.right, path)
        else:
            path.append(-1)
            randomSubTree(node.left, path)
    return path

# this function returns a subtree given a specific path
def getNode(node, path):
    if(path == []):
        return node
    if(path[0]==1):
        return getNode(node.right, path[1::])
    else:
        return getNode(node.left, path[1::])

#this function changes a node in one tree to another given node
def replaceNode(nodeA, pathA, nodeB):
    if(pathA == []):
        return nodeB
    elif(pathA[0] == 1):
        return Node(nodeA.val, nodeA.left, replaceNode(nodeA.right, pathA[1::], nodeB))
    elif(pathA[0] == -1):
        return Node(nodeA.val, replaceNode(nodeA.left, pathA[1::], nodeB), nodeA.right)

        

def crossover(treeA, treeB): 
    # determine a random node in each for crossover
    pathA = []
    pathB = []
    pathA = randomSubTree(treeA, pathA)
    pathB = randomSubTree(treeB, pathB)
    # swapping the random node in both trees
    temp = getNode(treeA, pathA)
    treeA = replaceNode(treeA, pathA, getNode(treeB, pathB))
    treeB = replaceNode(treeB, pathB, temp)
    return treeA, treeB
    
            
    

In [24]:
def find_pop(epochs = 200):

    population = []
    population_mse = []
    lowest_pop = None
    lowest_val = 10000000

    for i in range(10):
        tree = Node(random.choice(F))
        generate_tree(10, tree)
        population.append(tree)

    epochs = epochs

    for i in range(epochs):
        print("Epoch: " + str(i+1) + "/" + str(epochs))
        new_generation = []
        w = []
        j = 0
        pop_mse = 1000000
        pop_low = None
        for i in population:
            w.append(calculate_mse(x_train, y_train, i))
        total = sum(w)
    #     print(w,min(w))
        for i in range(len(w)):
            w[i] = (total-w[i])/total
        while(j < 10):
            operator = random.choices(['r','m','c'], weights = [2,5,6], k = 1)[0]
            if(operator == 'r'):
                child = random.choices(population, weights = w, k = 1)[0]
                new_generation.append(child)
                j+=1
            elif(operator == 'm'):
                child = random.choices(population, weights = w, k = 1)[0]
                child = mutate(child)
                new_generation.append(child)
                j+=1
            elif(operator == 'c'):
                children = random.choices(population, weights = w, k = 2)
                childA = children[0]
                childB = children[1]
                crossover(childA, childB)
                new_generation.append(childA)
                new_generation.append(childB)
                j+=2

        for i in new_generation:
            calc = calculate_mse(x_train,y_train,i)
            if calc < pop_mse:
                pop_low = i
                pop_mse = calc
                
        pop_high = None
        pop_high_mse = 0
        pop_high_index = None

        for i in range(len(population)):
            calc = calculate_mse(x_train,y_train,population[i])
            population_mse.append(calc)

            if calc > pop_high_mse:
                pop_high_mse = calc
                pop_high = population[i]
                pop_high_index = i
        population[pop_high_index] = pop_low
        population_mse[pop_high_index] = pop_mse

        print("Lowest Newly Generated MSE: "+ str(pop_mse))
        print("Lowest MSE Overall: " + str(min(population_mse)))
        print("Replaced MSE: " + str(pop_high_mse))

        for i in range(len(population)):
            calc = calculate_mse(x_train,y_train,population[i])
    #         print("Population " +str(i) + ": " +str(calc))
            if calc < lowest_val:
                lowest_val = calc
                lowest_pop = population[i]
        print(" ")
    return lowest_pop,population

In [25]:
low,pops = find_pop(25)

Epoch: 1/25
Lowest Newly Generated MSE: 174.16338010999723
Lowest MSE Overall: 174.16338010999723
Replaced MSE: 107945.30693331153
 
Epoch: 2/25
Lowest Newly Generated MSE: 229.9696001100007
Lowest MSE Overall: 174.16338010999723
Replaced MSE: 2145.0827842211056
 
Epoch: 3/25
Lowest Newly Generated MSE: 174.16338010999723
Lowest MSE Overall: 174.16338010999723
Replaced MSE: 1087.0998997766578
 
Epoch: 4/25
Lowest Newly Generated MSE: 174.16338010999723
Lowest MSE Overall: 174.16338010999723
Replaced MSE: 1052.1342681099886
 
Epoch: 5/25
Lowest Newly Generated MSE: 168.9560901100029
Lowest MSE Overall: 168.9560901100029
Replaced MSE: 950.667404110001
 
Epoch: 6/25
Lowest Newly Generated MSE: 168.9560901100029
Lowest MSE Overall: 168.9560901100029
Replaced MSE: 947.7128371100066
 
Epoch: 7/25
Lowest Newly Generated MSE: 168.9560901100029
Lowest MSE Overall: 168.9560901100029
Replaced MSE: 857.2005401099957
 
Epoch: 8/25
Lowest Newly Generated MSE: 174.16338010999723
Lowest MSE Overall: 1

In [26]:
print("Best performing on train set: " +str(calculate_mse(x_test,y_test,low)))

Best performing on train set: 110.40925471999967


In [27]:
for i in range(len(pops)):
    print("Population " +str(i) + ": " +str(calculate_mse(x_test,y_test,pops[i])))

Population 0: 110.40925471999967
Population 1: 110.40925471999967
Population 2: 110.40925471999967
Population 3: 110.40925471999967
Population 4: 110.40925471999967
Population 5: 110.40925471999967
Population 6: 110.40925471999967
Population 7: 110.40925471999967
Population 8: 110.40925471999967
Population 9: 110.40925471999967
